In [1]:
data_var = '2024-02-18'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7304,2024-02-18,Portugal Lfb Feminina,08:00,Galitos F,Barcelos F,2.20,1.62,135.5,1.86,1.80,1.5,2.06,1.68,z9sbbtxc,0.454545,0.617284,0.537634,0.555556,0.071829,239.836,139.994494,0.583709,1.2,1.643168,1.369306,247.00,4.044,2.617056,0.647145,5.0,234.268,60.929695,0.260085,1.2,1.643168,1.369306,221.40,3.518,0.874797,0.248663,-12.0,52,60,4.75,3.69,0.000,241.960,0.214724,0.023184,0.143690,0.00,0.000,inf,0.000000,0.0,0.000000,5.65,1.130,0.548673,0.000000,0.0,0.000000
7305,2024-02-18,Portugal Lfb Feminina,08:30,ACD Ferragudo F,Esgueira F,5.30,1.13,126.5,1.86,1.84,10.5,2.04,1.70,2ZiIgIjS,0.188679,0.884956,0.537634,0.543478,0.073635,181.440,52.632651,0.290083,0.6,1.341641,2.236068,111.24,3.282,1.071037,0.326337,-39.0,93.626,15.062011,0.160874,2.4,1.341641,0.559017,80.00,1.394,0.427352,0.306566,50.0,54,64,2.06,1.25,162.034,100.670,0.917149,0.007644,0.128565,-5.00,-1.000,-4.300000,0.000000,0.0,0.000000,-0.74,-0.148,-0.878378,0.000000,0.0,0.000000
7306,2024-02-18,Portugal Lfb Feminina,08:30,CAB Madeira F,Imortal F,1.99,1.74,131.5,1.88,1.79,1.5,1.85,1.85,YFyxAwy9,0.502513,0.574713,0.531915,0.558659,0.077225,217.774,87.853183,0.403414,0.0,0.000000,NaN,321.81,4.054,1.863982,0.459788,-94.0,84.840,21.587463,0.254449,1.8,1.643168,0.912871,116.25,1.334,0.131263,0.098398,-26.0,51,75,6.31,1.55,0.000,115.590,0.094786,0.034681,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,2.88,0.576,1.284722,0.000000,0.0,0.000000
7307,2024-02-18,Portugal Lfb Feminina,12:00,Uniao Sportiva F,Natacao F,1.02,11.10,131.5,1.84,1.86,-20.5,2.06,1.68,Yg01c0i3,0.980392,0.090090,0.543478,0.537634,0.070482,93.480,28.499330,0.304871,3.0,0.000000,0.000000,75.04,1.214,0.298463,0.245851,96.0,192.346,59.121417,0.307370,1.2,1.643168,1.369306,103.09,3.594,1.258463,0.350157,-45.0,67,61,1.12,1.69,84.606,187.162,1.176178,0.007644,0.143690,0.44,0.088,0.227273,0.000000,0.0,0.000000,-0.06,-0.012,-841.666667,0.000000,0.0,0.000000
7308,2024-02-18,Portugal Lfb Feminina,08:00,Benfica F,Gdessa Barreiro F,1.91,1.81,127.5,1.83,1.85,-1.5,2.10,1.65,GIk6dK69,0.523560,0.552486,0.546448,0.540541,0.076046,96.132,28.893828,0.300564,2.4,1.341641,0.559017,87.50,1.504,0.402964,0.267928,32.0,83.576,15.093950,0.180601,3.0,0.000000,0.000000,70.40,1.156,0.124218,0.107455,81.0,70,64,1.25,1.10,0.000,0.000,0.038016,0.007686,0.169706,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7452,2024-02-18,Eua Ncaa,16:00,Temple,UTSA Roadrunners,1.45,2.74,155.5,1.80,1.91,-5.5,1.80,1.91,Es93Asxh,0.689655,0.364964,0.555556,0.523560,0.054619,340.166,325.461553,0.956773,0.0,0.000000,NaN,918.00,4.936,4.816579,0.975806,-34.0,281.882,146.899205,0.521137,0.0,0.000000,NaN,463.40,4.014,2.117529,0.527536,-61.0,68,70,13.50,6.62,155.586,581.424,0.435402,0.041931,0.041931,-5.00,-1.000,-0.450000,0.493791,0.3,-0.193791,-5.00,-1.000,-1.740000,0.266271,0.2,-0.066271
7453,2024-02-18,Eua Ncaa,17:00,Illinois State,Evansville,1.59,2.35,136.5,1.80,1.91,-3.5,1.73,0.00,UsVfzR1i,0.628931,0.425532,0.555556,0.523560,0.054463,359.958,406.524552,1.129367,1.2,1.643168,1.369306

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7328,01:30,Japão B2.League,Fukushima,Shizuoka,2.34,Back Home
7332,11:00,Noruega Blno,Tromso,Ammerud,1.85,Back Home
7341,08:00,Suécia Superettan,RIG Mark,IK Eos,1.69,Back Home
7363,12:00,Suécia Superettan,KFUM Kalmar,Malbas,1.79,Back Home
7379,13:30,Espanha Leb Plata,Basquet L'Horta Godella,Salou,1.56,Back Home
7395,00:00,Eua Ncaa,San Jose State,Wyoming,1.80,Back Home
7401,11:00,Alemanha Pro A,Bayreuth,PS Karlsruhe,1.76,Back Home
7444,16:00,Eua Ncaa,East Carolina,Tulane,1.80,Back Home
